In [2]:
import numpy as np 
import pandas as pd
import nltk
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
from keras.regularizers import l2
import tensorflow as tf
import string 
import re 

In [3]:
train=pd.read_csv('C:/Users/ABC/Desktop/CVE_1.csv')

In [21]:
test=pd.read_csv('C:/Users/ABC/Desktop/CVE_2.csv')

In [4]:
# Import label encoder 
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
train['cvss_review']= label_encoder.fit_transform(train['cvss_review']) 
  
train['cvss_review'].unique() 

array([2, 0, 1])

In [5]:
train

,id,Unnamed: 0,mod_date,pub_date,cvss,cwe_code,cwe_name,summary,cvss_review
0,1,CVE-2019-16548,11/21/2019 15:15,11/21/2019 15:15,6.8,352,Cross-Site Request Forgery (CSRF),A cross-site request forgery vulnerability in ...,2
1,2,CVE-2019-16547,11/21/2019 15:15,11/21/2019 15:15,4.0,732,Incorrect Permission Assignment for Critical ...,Missing permission checks in various API endpo...,2
2,3,CVE-2019-16546,11/21/2019 15:15,11/21/2019 15:15,4.3,639,Authorization Bypass Through User-Controlled Key,Jenkins Google Compute Engine Plugin 4.1.1 and...,2
3,4,CVE-2013-2092,11/20/2019 21:22,11/20/2019 21:15,4.3,79,Improper Neutralization of Input During Web P...,Cross-site Scripting (XSS) in Dolibarr ERP/CRM...,2
4,5,CVE-2013-2091,11/20/2019 20:15,11/20/2019 20:15,7.5,89,Improper Neutralization of Special Elements u...,SQL injection vulnerability in Dolibarr ERP/CR...,0
...,...,...,...,...,...,...,...,...,...
1495,1496,CVE-2019-8084,10/28/2019 12:30,10/25/2019 15:15,4.3,79,Improper Neutralization of Input During Web P...,"Adobe Experience Manager versions 6.5, 6.4, 6....",2
1496,1497,CVE-2019-8083,10/28/2019 12:23,10/25/2019 15:15,4.3,79,Improper Neutralization of Input During Web P...,"Adobe Experience Manager versions 6.5, 6.4 and...",2
1497,1498,CVE-2019-17133,10/28/2019 2:15,10/4/2019 12:15,7.5,120,Buffer Copy without Checking Size of Input ('...,"In the Linux kernel through 5.3.2, cfg80211_mg...",0
1498,1499,CVE-2015-9503,10/28/2019 1:11,10/23/2019 17:15,4.3,79,Improper Neutralization of Input During Web P...,The Modern theme before 1.4.2 for WordPress ha...,2


In [6]:
train['summary'].apply(lambda x: " ".join(x.lower() for x in x.split()))


0       a cross-site request forgery vulnerability in ...
1       missing permission checks in various api endpo...
2       jenkins google compute engine plugin 4.1.1 and...
3       cross-site scripting (xss) in dolibarr erp/crm...
4       sql injection vulnerability in dolibarr erp/cr...
                              ...                        
1495    adobe experience manager versions 6.5, 6.4, 6....
1496    adobe experience manager versions 6.5, 6.4 and...
1497    in the linux kernel through 5.3.2, cfg80211_mg...
1498    the modern theme before 1.4.2 for wordpress ha...
1499    the auberge theme before 1.4.5 for wordpress h...
Name: summary, Length: 1500, dtype: object

In [7]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
train['summary'].apply(lambda x: [item for item in x if item not in stop_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ABC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0       [A,  , c, r, -, e,  , r, e, q, u, e,  , f, r, ...
1       [M, n, g,  , p, e, r, n,  , c, h, e, c, k,  , ...
2       [J, e, n, k, n,  , G, g, l, e,  , C, p, u, e, ...
3       [C, r, -, e,  , S, c, r, p, n, g,  , (, X, S, ...
4       [S, Q, L,  , n, j, e, c, n,  , v, u, l, n, e, ...
                              ...                        
1495    [A, b, e,  , E, x, p, e, r, e, n, c, e,  , M, ...
1496    [A, b, e,  , E, x, p, e, r, e, n, c, e,  , M, ...
1497    [I, n,  , h, e,  , L, n, u, x,  , k, e, r, n, ...
1498    [T, h, e,  , M, e, r, n,  , h, e, e,  , b, e, ...
1499    [T, h, e,  , A, u, b, e, r, g, e,  , h, e, e, ...
Name: summary, Length: 1500, dtype: object

In [8]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
train['summary'].apply(lambda x: [stemmer.stem(e) for e in x])

0       [a,  , c, r, o, s, s, -, s, i, t, e,  , r, e, ...
1       [m, i, s, s, i, n, g,  , p, e, r, m, i, s, s, ...
2       [j, e, n, k, i, n, s,  , g, o, o, g, l, e,  , ...
3       [c, r, o, s, s, -, s, i, t, e,  , s, c, r, i, ...
4       [s, q, l,  , i, n, j, e, c, t, i, o, n,  , v, ...
                              ...                        
1495    [a, d, o, b, e,  , e, x, p, e, r, i, e, n, c, ...
1496    [a, d, o, b, e,  , e, x, p, e, r, i, e, n, c, ...
1497    [i, n,  , t, h, e,  , l, i, n, u, x,  , k, e, ...
1498    [t, h, e,  , m, o, d, e, r, n,  , t, h, e, m, ...
1499    [t, h, e,  , a, u, b, e, r, g, e,  , t, h, e, ...
Name: summary, Length: 1500, dtype: object

In [9]:
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
train['summary'].apply(lambda x: [lemmatizer.lemmatize(e) for e in x])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ABC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0       [A,  , c, r, o, s, s, -, s, i, t, e,  , r, e, ...
1       [M, i, s, s, i, n, g,  , p, e, r, m, i, s, s, ...
2       [J, e, n, k, i, n, s,  , G, o, o, g, l, e,  , ...
3       [C, r, o, s, s, -, s, i, t, e,  , S, c, r, i, ...
4       [S, Q, L,  , i, n, j, e, c, t, i, o, n,  , v, ...
                              ...                        
1495    [A, d, o, b, e,  , E, x, p, e, r, i, e, n, c, ...
1496    [A, d, o, b, e,  , E, x, p, e, r, i, e, n, c, ...
1497    [I, n,  , t, h, e,  , L, i, n, u, x,  , k, e, ...
1498    [T, h, e,  , M, o, d, e, r, n,  , t, h, e, m, ...
1499    [T, h, e,  , A, u, b, e, r, g, e,  , t, h, e, ...
Name: summary, Length: 1500, dtype: object

In [31]:
from keras.preprocessing.text import Tokenizer

# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 1500
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train['summary'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 5954 unique tokens.


In [11]:
from keras.preprocessing.sequence import pad_sequences
X = tokenizer.texts_to_sequences(train['summary'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


Shape of data tensor: (1500, 250)


In [12]:
X

array([[  0,   0,   0, ..., 138,   2, 508],
       [  0,   0,   0, ..., 181,   8, 888],
       [  0,   0,   0, ...,   1, 584, 212],
       ...,
       [  0,   0,   0, ...,   3, 125, 129],
       [  0,   0,   0, ...,   1, 577, 148],
       [  0,   0,   0, ...,   1, 577, 148]])

In [65]:
Y=train.iloc[:,8]

In [66]:
Y.shape

(1500,)

In [15]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [67]:
from keras.layers import Embedding
from keras.layers.recurrent import LSTM
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 30


history = model.fit(X, Y, epochs=epochs, batch_size=100)


Epoch 1/30
1500/1500 [==============================] - 16s 11ms/step - loss: 1.0240 - acc: 0.5780
Epoch 2/30
1500/1500 [==============================] - 16s 10ms/step - loss: 0.8851 - acc: 0.6140
Epoch 3/30
1500/1500 [==============================] - 17s 11ms/step - loss: 0.8230 - acc: 0.6447
Epoch 4/30
1500/1500 [==============================] - 19s 12ms/step - loss: 0.7058 - acc: 0.6947
Epoch 5/30
1500/1500 [==============================] - 19s 12ms/step - loss: 0.5768 - acc: 0.7673
Epoch 6/30
1500/1500 [==============================] - 17s 11ms/step - loss: 0.4898 - acc: 0.8047
Epoch 7/30
1500/1500 [==============================] - 17s 11ms/step - loss: 0.3979 - acc: 0.8500
Epoch 8/30
1500/1500 [==============================] - 18s 12ms/step - loss: 0.3340 - acc: 0.8787
Epoch 9/30
1500/1500 [==============================] - 17s 11ms/step - loss: 0.2923 - acc: 0.8973
Epoch 10/30
1500/1500 [==============================] - 17s 12ms/step - loss: 0.2715 - acc: 0.9047
Epoch 11/

In [46]:
model.save('chatbot_model.h5', history)

In [47]:
from keras.models import load_model
model = load_model('chatbot_model.h5')


In [77]:
preceeding_events = ['Integer overflow in bufferobject.c in Python before 2.7.8 allows context-dependent attackers to obtain sensitive information from process memory via a large size and offset in a "buffer" function.']
seq = tokenizer.texts_to_sequences(preceeding_events)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
labels = ['high_risk','low_risk','medium_risk']
print(pred, labels[np.argmax(pred)])

[[5.5946753e-04 2.3951942e-04 9.9920100e-01]] medium_risk
